In [ ]:
import pandas as pd

In [ ]:
users = pd.read_csv("users.csv")
medicine = pd.read_csv("medicine.csv")
medicinedisease = pd.read_csv("medicinedisease.csv")
transaction_history = pd.read_csv("transaction_history.csv")
disease = pd.read_csv("disease.csv")

In [14]:
transaction_history

,timestamp,medicineId,amount,userId,price
0,2025-04-11 06:31:42,1,10,1,53284
1,2025-04-11 07:57:54,1,9,1,52297
2,2025-04-11 08:00:35,1,6,1,52016
3,2025-04-11 12:18:56,1,6,1,54813
4,2025-04-11 10:11:15,1,5,1,51383
...,...,...,...,...,...
8745,2025-04-05 07:21:40,50,8,5,47989
8746,2025-04-05 19:23:27,50,10,5,51700
8747,2025-04-05 04:11:44,50,5,5,44854
8748,2025-04-05 13:18:52,50,6,5,42730


In [ ]:
stock_projection